Title

Introduction:

    Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
    Clearly state the question you will try to answer with your project
    Identify and describe the dataset that will be used to answer the question



Preliminary exploratory data analysis:

    Demonstrate that the dataset can be read from the web into R 
    Clean and wrangle your data into a tidy format
    Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
    Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.



In [3]:
library(tidyverse)
library(repr)
library(tidymodels)
heart_scaled <- heart|>
 mutate(scaled_Cholesterol = scale(Cholesterol, center = TRUE),
        scaled_RestingBP = scale(RestingBP, center = TRUE),
        scaled_MaxHR = scale(MaxHR, center = TRUE))|>
select(HeartDisease, scaled_Cholesterol, scaled_RestingBP, scaled_MaxHR)
 
heart <- read_csv("https://raw.githubusercontent.com/josephsoo/dsci_100_group_12/main/data/heart.csv")|>
mutate(HeartDisease = as_factor(HeartDisease))
head(heart)

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Learn how to get started at https://www.tidymodels.org/start/

Rows: 918 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): Sex, ChestPainType, RestingECG, ExerciseAngina, ST_Slope
dbl (7): Age, RestingBP, Cholesterol, FastingBS, MaxHR, Oldpeak, HeartDisease

ℹ Use

Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<fct>
40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0


In [ ]:
heart_scaled <- heart|>
 mutate(scaled_Cholesterol = scale(Cholesterol, center = TRUE),
        scaled_RestingBP = scale(RestingBP, center = TRUE),
        scaled_MaxHR = scale(MaxHR, center = TRUE))|>
select(HeartDisease, scaled_Cholesterol, scaled_RestingBP, scaled_MaxHR)

In [6]:
set.seed(3456) 

heart_split <- initial_split(heart_scaled, prop = 0.75, strata = HeartDisease)  
heart_train <- training(heart_split)   
heart_test <- testing(heart_split)

Methods:

    Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
    Describe at least one way that you will visualize the results



FOR THE METHOD: do something like 6.8.2 in the textbook (https://datasciencebook.ca/classification2.html#finding-a-good-subset-of-predictors) to find the best subset of predictors with our set of resting bp, cholesterol, and age. Of course do other stuff too...

                                                         

Expected outcomes and significance:

**What do you expect to find?**
- In this project, we exepect to be able to classify
**What impact could such findings have?**
**What future questions could this lead to?**

